In [16]:
import sys
import csv
from collections import defaultdict
from collections import Counter
import geopandas as gpd
import datetime as dt
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.ticker as plticker
import os

csv.field_size_limit(sys.maxsize)


9223372036854775807

In [17]:
def true_date(date):
    
    year = str(date).split(".")[0]
    decimal = f'0.{str(date).split(".")[1]}'
    
    if year == '2020':
        day_no = float(decimal) * 366
    else:
        day_no = float(decimal) * 365
        
    delta = dt.timedelta(day_no)
    
    start = dt.date(year=int(year), month=1, day=1)
    tr_date = start + delta
    
    return tr_date

def decimal_date(date):
    
    year = date.year
    if year == 2020:
        div = 366
    else:
        div = 365
    start = dt.date(year=year, month=1, day=1)
    decimal = (date - start).days/div
    
    dec_date = year + decimal
    return dec_date

## find large intros

In [18]:
intro_data_dir = "TREE_FILES"
lineage_sizes = {}
for intro_data in os.listdir(intro_data_dir):
    if intro_data.endswith(".tsv"):
        tree_number = intro_data.split(".")[0].split("_")[1][-1]
        with open(os.path.join(intro_data_dir,intro_data)) as f:
            data = csv.DictReader(f, delimiter="\t")
            for line in data:
                lineage_sizes[f'{tree_number}_{line["lineage"]}'] = int(line['ntaxa'])
                

sorted_sizes = {k: v for k, v in sorted(lineage_sizes.items(), key=lambda item: item[1], reverse=True)}  
sorted_sizes  

{'0_92': 12334,
 '0_190': 3593,
 '2_337': 3214,
 '0_67': 1872,
 '1_51': 1785,
 '0_322': 1621,
 '0_386': 1564,
 '0_239': 1496,
 '1_26': 1190,
 '0_316': 1043,
 '0_204': 984,
 '2_167': 957,
 '0_381': 839,
 '0_248': 837,
 '1_357': 810,
 '0_355': 793,
 '0_79': 659,
 '0_260': 606,
 '0_304': 585,
 '2_437': 518,
 '2_431': 487,
 '0_261': 452,
 '2_88': 422,
 '2_396': 402,
 '0_378': 387,
 '0_372': 318,
 '1_56': 318,
 '1_308': 302,
 '1_218': 298,
 '0_407': 286,
 '1_277': 284,
 '1_10': 269,
 '0_293': 268,
 '1_311': 263,
 '0_74': 237,
 '0_309': 234,
 '0_64': 232,
 '1_105': 232,
 '1_90': 226,
 '2_530': 222,
 '1_314': 217,
 '0_368': 216,
 '2_309': 211,
 '0_95': 203,
 '0_259': 202,
 '1_306': 201,
 '0_342': 195,
 '0_51': 169,
 '2_370': 163,
 '0_333': 153,
 '0_335': 150,
 '1_347': 150,
 '1_369': 149,
 '2_429': 148,
 '0_151': 138,
 '1_142': 138,
 '2_335': 135,
 '0_97': 134,
 '1_304': 127,
 '0_266': 125,
 '2_332': 124,
 '2_404': 122,
 '0_421': 120,
 '1_321': 116,
 '0_222': 107,
 '0_103': 102,
 '1_180': 95,

In [5]:
large_intros = ["0_92", "0_190", "2_337", "0_67", "1_51", "0_322", "0_386"] #all over 1500 sequences - seven of them 

In [6]:
total = 0
count = 0
for k,i in sorted_sizes.items():
    if k not in large_intros:
        if i >= 5:
            count += 1
            total += i
print(total)
print(count)

25394
280


In [7]:
large_intro_count = 0
for i in large_intros:
    large_intro_count += sorted_sizes[i]
print(large_intro_count)

25983


In [6]:
total = 0
for i in sorted_sizes.values():
    total += i
    
print(total)

52992


In [30]:
lin_to_taxa = defaultdict(list)
lin_to_tmrca = {}

intro_data_dir = "../input_files/median_import_tree/"
for intro_data in os.listdir(intro_data_dir):
    if intro_data.endswith(".tsv"):
        tree_number = intro_data.split(".")[0].split("_")[1][-1]
        with open(os.path.join(intro_data_dir,intro_data)) as f:
            data = csv.DictReader(f, delimiter="\t")
            for line in data:
                lin_to_tmrca[f'{tree_number}_{line["lineage"]}'] = line['tmrca'].split("T")[0]
        #         if line['lineage'] in wanted_intros:
                taxa = line['taxa'].split(";")
                for i in taxa:
                    lin_to_taxa[f'{tree_number}_{line["lineage"]}'].append(i.strip(" "))

## investigating size distribution of top 20%

In [36]:
print(len(lin_to_taxa))
print(len(lin_to_taxa)*0.2)
lins_to_counts = {}
for k,v in lin_to_taxa.items():
    lins_to_counts[k] = len(v)

sorted_lins = {k: v for k, v in sorted(lins_to_counts.items(), key=lambda item: item[1], reverse=True)}    

1457
291.40000000000003


In [37]:
top_20pc = []
top_20pc_names = []
top_5 = []
all_counts = []
count = 0
for k,v in sorted_lins.items():
    count += 1
    if count <= 291:
        top_20pc.append(v)
        top_20pc_names.append(k)
    all_counts.append(v)
    if count <=5:
        top_5.append(v)

In [38]:
print(len(top_20pc))

291


In [39]:
print(sum(top_20pc))
print(sum(all_counts))

51393
52992


In [40]:
print(sum(top_20pc)/sum(all_counts)*100)
print(sum(top_5))

96.98256340579711
22798


## get numbers of sequences in each country

In [12]:
metadata = "../input_files/cog_2021-07-14_metadata.csv"

end = dt.date(2021,6,15)

places = []
with open(metadata) as f:
    data = csv.DictReader(f)
    print(data.fieldnames)
    for line in data:
        if line["country"] == "UK":
            if line["scorpio_call"] == "Delta (B.1.617.2-like)" and dt.datetime.strptime(line["sample_date"],"%Y-%m-%d").date() <= end:
                places.append(line["adm1"])
            
            

['sequence_name', 'secondary_identifier', 'sample_date', 'epi_week', 'country', 'adm1', 'adm2', 'outer_postcode', 'is_surveillance', 'is_community', 'is_hcw', 'is_travel_history', 'travel_history', 'lineage', 'lineages_version', 'lineage_conflict', 'lineage_ambiguity_score', 'scorpio_call', 'scorpio_support', 'scorpio_conflict']


In [13]:
print(len(places))

68722


In [14]:
place_counts = Counter(places)

In [15]:
place_counts

Counter({'UK-ENG': 57592, 'UK-SCT': 9738, 'UK-WLS': 1067, 'UK-NIR': 325})